In [3]:
pip install streamlit-jupyter

  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.8.1
    Uninstalling ipywidgets-7.8.1:
      Successfully uninstalled ipywidgets-7.8.1
Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
import json
import pandas as pd

# List to store property data for DataFrame
property_records = []

def search_properties(api_key, location, property_type="house", limit=3):
    """
    Search for properties using the Realtor API
    
    Parameters:
        api_key (str): Your RapidAPI key
        location (str): City and state, e.g., "New York, NY"
        property_type (str): Type of property (house, condo, apartment)
        limit (int): Maximum number of results to return
    
    Returns:
        dict: JSON response from the API
    """
    url = 'https://realtor16.p.rapidapi.com/search/forsale'
    
    # Parse city and state from location
    #city, state = [x.strip() for x in location.split(',')]
    
    #querystring = {
    #    "city": city,
    #    "state_code": state,
    #    "limit": str(limit),
    #    #"offset": "0",
    #    #"sort": "relevance",
    #    #"property_type": property_type
    #}

    querystring =  {"location":location, 
                    "search_radius":"0",
                    "limit":limit,
                    "property_type":property_type}
    
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor16.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def display_and_store_properties(properties_data):
    """Display property information in a readable format based on the actual JSON structure"""
    if not properties_data or 'properties' not in properties_data:
        print("No properties found or invalid data")
        return
    
    for prop in properties_data['properties']:
        print(f"\n{'='*50}")
        
        # Address information
        address = prop.get('location', {}).get('address', {})
        address_line = address.get('line', 'N/A')
        city = address.get('city', 'N/A')
        state = address.get('state_code', 'N/A')
        zip_code = address.get('postal_code', 'N/A')
        
        print(f"Address: {address_line}")
        print(f"City: {city}")
        print(f"State: {state}")
        print(f"Zip: {zip_code}")
        
        # Property details
        description = prop.get('description', {})
        price = prop.get('list_price', 'N/A')
        beds = description.get('beds', 'N/A')
        baths = description.get('baths_consolidated', 'N/A')
        sqft = description.get('sqft', 'N/A')
        lot_sqft = description.get('lot_sqft', 'N/A')
        property_type_desc = description.get('type', 'N/A')
        
        if price != 'N/A':
            print(f"Price: ${price:,}")
        else:
            print(f"Price: {price}")
        print(f"Beds: {beds}")
        print(f"Baths: {baths}")
        print(f"Sq Ft: {sqft}")
        print(f"Lot Size (sq ft): {lot_sqft}")
        print(f"Property Type: {property_type_desc}")
        
        
        # Status flags
        flags = prop.get('flags', {})
        status_flags = []
        if flags.get('is_new_listing'):
            status_flags.append("NEW LISTING")
        if flags.get('is_price_reduced'):
            status_flags.append("PRICE REDUCED")
        if flags.get('is_pending'):
            status_flags.append("PENDING")
        if flags.get('is_foreclosure'):
            status_flags.append("FORECLOSURE")
        if flags.get('is_coming_soon'):
            status_flags.append("COMING SOON")
        if flags.get('is_new_construction'):
            status_flags.append("NEW CONSTRUCTION")
        if flags.get('is_contingent'):
            status_flags.append("CONTINGENT")
            
        status = ', '.join(status_flags) if status_flags else "ACTIVE"
        print(f"Status: {status}")
        
        # Listing information
        listing_id = prop.get('listing_id', 'N/A')
        property_id = prop.get('property_id', 'N/A')
        list_date = prop.get('list_date', 'N/A')
        
        print(f"Listing ID: {listing_id}")
        print(f"Property ID: {property_id}")
        print(f"List Date: {list_date}")
        
        # Images
        primary_image = prop.get('primary_photo', {}).get('href', 'N/A')
        if primary_image != 'N/A':
            print(f"Primary Image: {primary_image}")
        
        # Additional photos count
        photos = prop.get('photos', [])
        additional_photos = len(photos) - 1 if photos and len(photos) > 1 else 0
        if additional_photos > 0:
            print(f"Additional Photos: {additional_photos}")
        
        # Virtual tours
        virtual_tours = prop.get('virtual_tours', [])
        virtual_tour = virtual_tours[0].get('href', 'N/A') if virtual_tours and len(virtual_tours) > 0 else 'N/A'
        if virtual_tour != 'N/A':
            print(f"Virtual Tour: {virtual_tour}")
        
        # Listing agency
        branding = prop.get('branding', [])
        listed_by = branding[0].get('name', 'N/A') if branding and len(branding) > 0 else 'N/A'
        if listed_by != 'N/A':
            print(f"Listed By: {listed_by}")
        
        # Permalink
        permalink = prop.get('permalink', 'N/A')
        listing_url = f"https://www.realtor.com/realestateandhomes-detail/{permalink}" if permalink != 'N/A' else 'N/A'
        print(f"Listing URL: {listing_url}")
        
        # Add data to records list
        property_records.append({
            "Address": address_line,
            "City": city,
            "State": state,
            "Zip": zip_code,
            "Price": price,
            "Beds": beds,
            "Baths": baths,
            "Sq Ft": sqft,
            "Lot Size (sq ft)": lot_sqft,
            "Property Type": property_type_desc,
            "Status": status,
            "Listing ID": listing_id,
            "Property ID": property_id,
            "List Date": list_date,
            "Primary Image": primary_image,
            "Additional Photos": additional_photos,
            "Virtual Tour": virtual_tour,
            "Listed By": listed_by,
            "Listing URL": listing_url
        })
    
    # Create DataFrame from records
    df = pd.DataFrame(property_records)
    
    print(f"\n{'='*50}")
    print(f"Found {len(property_records)} properties.")
    print("Data stored in DataFrame")
    
    # Display DataFrame overview
    print("\nDataFrame Preview:")
    print(f"Shape: {df.shape}")
    print(df.head())
    
    return df


if __name__ == "__main__":
    # Replace with your API key
    API_KEY = "f8652a60a8mshf167b4d824fc77bp1b9522jsnb4348a0da4d6"

    # Make sure pandas is installed
    try:
        import pandas as pd
    except ImportError:
        print("Pandas is not installed. Installing now...")
        import subprocess
        subprocess.check_call(["pip", "install", "pandas"])
        import pandas as pd
    
    # Example search
    location = input("Enter city and state (e.g., Miami, FL): ")
    property_type = input("Enter property type (house, condo, apartment): ") or "house"
    limit = input("Enter maximum number of results (default 10): ") or 10
    
    print(f"\nSearching for {property_type}s in {location}...")
    results = search_properties(API_KEY, location, property_type, int(limit))
    
    if results:
        # Store results in DataFrame and display
        df = display_and_store_properties(results)

Enter city and state (e.g., Miami, FL):  63122
Enter property type (house, condo, apartment):  house
Enter maximum number of results (default 10):  3



Searching for houses in 63122...

Address: 1707 Hoffman Ave
City: Saint Louis
State: MO
Zip: 63122
Price: $775,000
Beds: 2
Baths: 2
Sq Ft: 2135
Lot Size (sq ft): 24394
Property Type: single_family
Status: NEW LISTING, COMING SOON
Listing ID: 2980744708
Property ID: 8202811860
List Date: 2025-04-18T19:37:05.000000Z
Primary Image: https://ap.rdcpix.com/f21797638384b7f561c12cc5ccd636b1l-m293010275s.jpg
Additional Photos: 1
Virtual Tour: https://brokerageengine.s3.amazonaws.com/listing-media/dielmannsir/1744996999026_1707_Hoffman_Ave_AP2_Undranded.mp4
Listed By: Dielmann Sotheby's International Realty
Listing URL: https://www.realtor.com/realestateandhomes-detail/1707-Hoffman-Ave_Saint-Louis_MO_63122_M82028-11860

Address: 2539 Saint Giles Rd
City: Saint Louis
State: MO
Zip: 63122
Price: $535,000
Beds: 4
Baths: 2.5
Sq Ft: 2723
Lot Size (sq ft): 21018
Property Type: single_family
Status: NEW LISTING, COMING SOON
Listing ID: 2980797132
Property ID: 7409130040
List Date: 2025-04-21T14:26:54.

In [12]:
df

,Address,City,State,Zip,Price,Beds,Baths,Sq Ft,Lot Size (sq ft),Property Type,Status,Listing ID,Property ID,List Date,Primary Image,Additional Photos,Virtual Tour,Listed By,Listing URL
0,1707 Hoffman Ave,Saint Louis,MO,63122,775000,2,2,2135,24394,single_family,"NEW LISTING, COMING SOON",2980744708,8202811860,2025-04-18T19:37:05.000000Z,https://ap.rdcpix.com/f21797638384b7f561c12cc5...,1,https://brokerageengine.s3.amazonaws.com/listi...,Dielmann Sotheby's International Realty,https://www.realtor.com/realestateandhomes-det...
1,2539 Saint Giles Rd,Saint Louis,MO,63122,535000,4,2.5,2723,21018,single_family,"NEW LISTING, COMING SOON",2980797132,7409130040,2025-04-21T14:26:54.000000Z,https://ap.rdcpix.com/b111d4392fd8654a17fa127d...,1,N/A,Coldwell Banker Realty - Gundaker,https://www.realtor.com/realestateandhomes-det...
2,1017 Plaza Ter,Saint Louis,MO,63122,289000,3,1,984,6251,single_family,"NEW LISTING, COMING SOON",2980811750,7927719747,2025-04-21T19:36:52.000000Z,https://ap.rdcpix.com/b6edc27ac81ff564e1465b12...,1,N/A,Laura McCarthy Real Estate,https://www.realtor.com/realestateandhomes-det...


In [1]:
import requests
import json
import pandas as pd

def search_rental_properties(api_key, location, min_price=None, max_price=None, beds=None, baths=None, limit=10):
    """
    Search for rental properties using the Realtor API
    
    Parameters:
        api_key (str): Your RapidAPI key
        location (str): City and state, e.g., "Kirkwood, MO"
        min_price (int, optional): Minimum rental price
        max_price (int, optional): Maximum rental price
        beds (int, optional): Number of bedrooms
        baths (int, optional): Number of bathrooms
        limit (int, optional): Maximum number of results to return
    
    Returns:
        dict: JSON response from the API
    """
    url = "https://realtor16.p.rapidapi.com/search/forrent"
    
    # Parse city and state from location
    #city, state = [x.strip() for x in location.split(',')]
    
    #querystring = {
    #    "city": city,
    #    "state_code": state,
    #    "limit": str(limit),
    #    "offset": "0",
    #    "sort": "relevance"
    #}

    querystring =  {"location":location, 
                    "search_radius":"0",
                    "limit":limit
                    #"property_type":property_type
                   }

    # Add optional filters if provided
    if min_price:
        querystring["price_min"] = str(min_price)
    if max_price:
        querystring["price_max"] = str(max_price)
    if beds:
        querystring["beds_min"] = str(beds)
    if baths:
        querystring["baths_min"] = str(baths)
    
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor16.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def display_and_store_rentals(properties_data):
    """Display rental property information and store in a pandas DataFrame"""
    if not properties_data or 'properties' not in properties_data:
        print("No rental properties found or invalid data")
        return None
    
    # List to store property data for DataFrame
    property_records = []
    
    # Process each property
    for prop in properties_data['properties']:
        try:
            print(f"\n{'='*50}")
            
            # Address information
            address = prop.get('location', {}).get('address', {})
            address_line = address.get('line', 'N/A')
            city = address.get('city', 'N/A')
            state = address.get('state_code', 'N/A')
            zip_code = address.get('postal_code', 'N/A')
            
            print(f"Address: {address_line}")
            print(f"City: {city}")
            print(f"State: {state}")
            print(f"Zip: {zip_code}")
            
            # Property details
            description = prop.get('description', {})
            rent = prop.get('list_price')
            beds = description.get('beds', 'N/A')
            baths = description.get('baths_consolidated', 'N/A')
            sqft = description.get('sqft', 'N/A')
            property_type_desc = description.get('type', 'N/A')
            sub_type = description.get('sub_type', 'N/A')
            
            # Error handling for rent formatting
            if rent is not None:
                try:
                    print(f"Monthly Rent: ${rent:,}")
                    formatted_rent = f"${rent:,}"
                    rent = rent
                except (TypeError, ValueError):
                    print(f"Monthly Rent: ${rent}")
                    formatted_rent = f"${rent}"
                    rent = rent
            else:
                print("Monthly Rent: N/A")
                formatted_rent = "N/A"
                rent = "N/A"
                
            print(f"Beds: {beds}")
            print(f"Baths: {baths}")
            print(f"Sq Ft: {sqft}")
            print(f"Property Type: {property_type_desc}")
            print(f"Sub Type: {sub_type}")
            
            # Status flags
            flags = prop.get('flags', {})
            status_flags = []
            if flags and isinstance(flags, dict):
                if flags.get('is_new_listing'):
                    status_flags.append("NEW LISTING")
                if flags.get('is_pending'):
                    status_flags.append("PENDING")
                
            status = ', '.join(status_flags) if status_flags else "ACTIVE"
            print(f"Status: {status}")
            
            # Price reduction info
            price_reduced = prop.get('price_reduced_amount')
            if price_reduced is not None:
                try:
                    print(f"Price Reduced: ${price_reduced:,}")
                except (TypeError, ValueError):
                    print(f"Price Reduced: ${price_reduced}")
            
            # Listing information
            listing_id = prop.get('listing_id', 'N/A')
            property_id = prop.get('property_id', 'N/A')
            list_date = prop.get('list_date', 'N/A')
            
            print(f"Listing ID: {listing_id}")
            print(f"Property ID: {property_id}")
            if list_date:
                print(f"List Date: {list_date}")
            
            # Pet policy with error handling
            pet_policy = prop.get('pet_policy', {})
            pets_allowed = []
            
            try:
                if pet_policy and isinstance(pet_policy, dict):
                    if pet_policy.get('cats'):
                        pets_allowed.append("Cats")
                    if pet_policy.get('dogs_small'):
                        pets_allowed.append("Small Dogs")
                    if pet_policy.get('dogs_large'):
                        pets_allowed.append("Large Dogs")
                
                pets_string = ', '.join(pets_allowed) if pets_allowed else "No information"
                print(f"Pets Allowed: {pets_string}")
            except Exception as e:
                pets_string = "Error retrieving pet information"
                print(f"Pets Allowed: Error ({str(e)})")
            
            # Get security deposit info from details if available
            security_deposit = "N/A"
            availability_date = "N/A"
            
            try:
                details = prop.get('details', [])
                if details and isinstance(details, list):
                    for detail in details:
                        if not isinstance(detail, dict):
                            continue
                            
                        category = detail.get('category')
                        texts = detail.get('text', [])
                        
                        if not isinstance(texts, list):
                            continue
                            
                        if category == "Rental Info":
                            for text in texts:
                                if isinstance(text, str) and "Security Deposit:" in text:
                                    security_deposit = text.split("Security Deposit:")[1].strip()
                        
                        if category == "Other Property Info":
                            for text in texts:
                                if isinstance(text, str) and "Availability Date:" in text:
                                    availability_date = text.split("Availability Date:")[1].strip()
            except Exception as e:
                print(f"Error processing property details: {str(e)}")
                
            if security_deposit != "N/A":
                print(f"Security Deposit: ${security_deposit}")
            if availability_date != "N/A":
                print(f"Available From: {availability_date}")
            
            # Images
            primary_image = "N/A"
            try:
                primary_photo = prop.get('primary_photo', {})
                if primary_photo and isinstance(primary_photo, dict):
                    primary_image = primary_photo.get('href', 'N/A')
                
                if primary_image != 'N/A':
                    print(f"Primary Image: {primary_image}")
            except Exception as e:
                print(f"Error processing image: {str(e)}")
            
            # Additional photos count
            additional_photos = 0
            try:
                photos = prop.get('photos', [])
                if photos and isinstance(photos, list):
                    additional_photos = max(0, len(photos) - 1)
                
                if additional_photos > 0:
                    print(f"Additional Photos: {additional_photos}")
            except Exception as e:
                print(f"Error counting photos: {str(e)}")
                additional_photos = "Error"
            
            # Virtual tours
            virtual_tour = "N/A"
            try:
                virtual_tours = prop.get('virtual_tours', [])
                if virtual_tours and isinstance(virtual_tours, list) and len(virtual_tours) > 0:
                    if isinstance(virtual_tours[0], dict):
                        virtual_tour = virtual_tours[0].get('href', 'N/A')
                
                if virtual_tour != 'N/A':
                    print(f"Virtual Tour: {virtual_tour}")
            except Exception as e:
                print(f"Error processing virtual tour: {str(e)}")
            
            # Contact info
            contact_phone = "N/A"
            try:
                advertisers = prop.get('advertisers', [])
                if advertisers and isinstance(advertisers, list):
                    for advertiser in advertisers:
                        if not isinstance(advertiser, dict):
                            continue
                            
                        if advertiser.get('type') == "management" and advertiser.get('office'):
                            office = advertiser.get('office')
                            if isinstance(office, dict):
                                phones = office.get('phones', [])
                                if phones and isinstance(phones, list) and len(phones) > 0:
                                    if isinstance(phones[0], dict):
                                        contact_phone = phones[0].get('number', 'N/A')
                
                if contact_phone != 'N/A':
                    print(f"Contact Phone: {contact_phone}")
            except Exception as e:
                print(f"Error processing contact info: {str(e)}")
            
            # Permalink
            permalink = prop.get('permalink', 'N/A')
            listing_url = "N/A"
            try:
                if permalink != 'N/A':
                    listing_url = f"https://www.realtor.com/apartments/{permalink}"
                    print(f"Listing URL: {listing_url}")
            except Exception as e:
                print(f"Error creating listing URL: {str(e)}")
            
            # Add data to records list
            try:
                property_records.append({
                    "Address": address_line,
                    "City": city,
                    "State": state,
                    "Zip": zip_code,
                    "Monthly Rent": formatted_rent,
                    "Monthly Rent v2": rent,
                    "Beds": beds,
                    "Baths": baths,
                    "Sq Ft": sqft,
                    "Property Type": property_type_desc,
                    "Sub Type": sub_type,
                    "Status": status,
                    "Security Deposit": security_deposit,
                    "Available From": availability_date,
                    "Pets Allowed": pets_string,
                    "Listing ID": listing_id,
                    "Property ID": property_id,
                    "List Date": list_date,
                    "Contact Phone": contact_phone,
                    "Primary Image": primary_image,
                    "Additional Photos": additional_photos,
                    "Virtual Tour": virtual_tour,
                    "Listing URL": listing_url
                })
            except Exception as e:
                print(f"Error adding property to records: {str(e)}")
        
        except Exception as e:
            print(f"Error processing property: {str(e)}")
            continue
    
    # Create DataFrame from records
    try:
        df = pd.DataFrame(property_records)
        
        print(f"\n{'='*50}")
        print(f"Found {len(property_records)} rental properties.")
        print("Data stored in DataFrame")
        
        # Display DataFrame overview
        print("\nDataFrame Preview:")
        print(f"Shape: {df.shape}")
        print(df.head())
        
        return df
    except Exception as e:
        print(f"Error creating DataFrame: {str(e)}")
        return None


if __name__ == "__main__":
    # Replace with your API key
    API_KEY = "f8652a60a8mshf167b4d824fc77bp1b9522jsnb4348a0da4d6"
    
    # Make sure pandas is installed
    try:
        import pandas as pd
    except ImportError:
        print("Pandas is not installed. Installing now...")
        import subprocess
        subprocess.check_call(["pip", "install", "pandas"])
        import pandas as pd
    
    # Rental search parameters
    location = input("Enter city and state (e.g., Kirkwood, MO): ")
    
    # Optional parameters
    min_rent = input("Enter minimum rent (or press Enter to skip): ")
    min_rent = int(min_rent) if min_rent.strip() else None
    
    max_rent = input("Enter maximum rent (or press Enter to skip): ")
    max_rent = int(max_rent) if max_rent.strip() else None
    
    beds = input("Enter minimum number of bedrooms (or press Enter to skip): ")
    beds = int(beds) if beds.strip() else None
    
    baths = input("Enter minimum number of bathrooms (or press Enter to skip): ")
    baths = int(baths) if baths.strip() else None
    
    limit = input("Enter maximum number of results (default 10): ") or 10
    
    print(f"\nSearching for rentals in {location}...")
    results = search_rental_properties(
        API_KEY, 
        location, 
        min_price=min_rent, 
        max_price=max_rent, 
        beds=beds, 
        baths=baths, 
        limit=int(limit)
    )
    
    if results:
        # Store results in DataFrame and display
        df_rent = display_and_store_rentals(results)
        
        # Now you can work with the DataFrame
        print("\nYou can now work with the DataFrame 'df'")

Enter city and state (e.g., Kirkwood, MO):  63122
Enter minimum rent (or press Enter to skip):  
Enter maximum rent (or press Enter to skip):  
Enter minimum number of bedrooms (or press Enter to skip):  
Enter minimum number of bathrooms (or press Enter to skip):  
Enter maximum number of results (default 10):  100



Searching for rentals in 63122...

Address: 431 Meacham St
City: Kirkwood
State: MO
Zip: 63122
Monthly Rent: $1,550
Beds: 2
Baths: 1
Sq Ft: 896
Property Type: single_family
Sub Type: None
Status: NEW LISTING
Listing ID: 2980653081
Property ID: 8586312525
List Date: 2025-04-16T20:47:04.000000Z
Pets Allowed: Cats
Available From: 2025-06-15
Primary Image: https://ap.rdcpix.com/13df560fad88e4d35fd7ea820b9fea0cl-m1644206318s.jpg
Additional Photos: 1
Listing URL: https://www.realtor.com/apartments/431-Meacham-St_Saint-Louis_MO_63122_M85863-12525

Address: 479 S Holmes Ave Apt C
City: Kirkwood
State: MO
Zip: 63122
Monthly Rent: $1,099
Beds: 2
Baths: 1
Sq Ft: 750
Property Type: condos
Sub Type: condo
Status: ACTIVE
Price Reduced: $100
Listing ID: 2964922541
Property ID: 9720737248
Pets Allowed: Cats, Small Dogs, Large Dogs
Security Deposit: $1099.00
Available From: 2024-05-01
Primary Image: https://ap.rdcpix.com/910b1fd35e35ef9709733ed595e909d7l-m363626523s.jpg
Additional Photos: 1
Contact Ph

In [7]:
df_rent

,Address,City,State,Zip,Monthly Rent,Monthly Rent v2,Beds,Baths,Sq Ft,Property Type,...,Available From,Pets Allowed,Listing ID,Property ID,List Date,Contact Phone,Primary Image,Additional Photos,Virtual Tour,Listing URL
0,431 Meacham St,Kirkwood,MO,63122,"$1,550",1550,2.0,1,896.0,single_family,...,2025-06-15,Cats,2980653081,8586312525,2025-04-16T20:47:04.000000Z,N/A,https://ap.rdcpix.com/13df560fad88e4d35fd7ea82...,1,N/A,https://www.realtor.com/apartments/431-Meacham...
1,479 S Holmes Ave Apt C,Kirkwood,MO,63122,"$1,099",1099,2.0,1,750.0,condos,...,2024-05-01,"Cats, Small Dogs, Large Dogs",2964922541,9720737248,None,3147822225,https://ap.rdcpix.com/910b1fd35e35ef9709733ed5...,1,N/A,https://www.realtor.com/apartments/479-S-Holme...
2,437 S Clay Ave Apt 7,Saint Louis,MO,63122,"$1,495",1495,2.0,1,860.0,condos,...,N/A,No information,2977432216,7398601968,2025-01-20T22:03:40.000000Z,N/A,https://ap.rdcpix.com/f93633179205cf4c26cc57cd...,1,N/A,https://www.realtor.com/apartments/437-S-Clay-...
3,1943 Greenpoint Dr Apt 301,Saint Louis,MO,63122,"$1,095",1095,1.0,1,657.0,condos,...,2025-04-18,Cats,2980562251,9436956425,2025-04-14T20:16:51.000000Z,N/A,https://ap.rdcpix.com/fc1665e31edd52a53cb3ba75...,1,N/A,https://www.realtor.com/apartments/1943-Greenp...
4,1722 W Woodbine Ave,Kirkwood,MO,63122,"$1,395",1395,2.0,1,898.0,single_family,...,2023-08-31,No information,2980128031,7254029187,2025-04-03T12:26:40.000000Z,N/A,https://ap.rdcpix.com/29028d94bd3e57413fd255ff...,1,N/A,https://www.realtor.com/apartments/1722-W-Wood...
5,1968 Greenglen Dr Unit 1968-202,Kirkwood,MO,63122,"$1,675",1675,2.0,2,990.0,condos,...,2025-01-26,No information,2977632261,9282220308,2025-01-27T08:18:33.000000Z,3145560938,https://ap.rdcpix.com/cf5090a7de38746e8fa27566...,1,N/A,https://www.realtor.com/apartments/1968-Greeng...
6,411 S Geyer Rd Apt 9B,Saint Louis,MO,63122,"$1,595",1595,2.0,1,1000.0,apartment,...,2025-06-06,"Cats, Small Dogs",2980040130,9652969468,2025-04-01T21:23:37.000000Z,3146067333,https://ap.rdcpix.com/78456eaff49b8f5e533578dc...,1,N/A,https://www.realtor.com/apartments/411-S-Geyer...
7,1 Provincial Ct,Kirkwood,MO,63122,N/A,N/A,NaN,None,NaN,apartment,...,N/A,Cats,2977155698,8668074003,2025-01-03T15:16:41.000000Z,None,https://ar.rdcpix.com/5a6c1cfb6c67a5d841af7d04...,1,https://vimeo.com/1076827579?share=copy#t=0,https://www.realtor.com/apartments/1-Provincia...
8,419 S Clay Ave,Kirkwood,MO,63122,"$2,570",2570,3.0,1.5,1393.0,single_family,...,N/A,"Cats, Small Dogs, Large Dogs",2979904402,7049476429,2025-03-28T18:05:34.000000Z,8886599596,https://ap.rdcpix.com/301f53eb813ec6c11ae7d3bb...,1,N/A,https://www.realtor.com/apartments/419-S-Clay-...
9,10824 Big Bend Rd,Kirkwood,MO,63122,N/A,N/A,NaN,None,NaN,apartment,...,N/A,Cats,2933719371,9676310554,2021-08-31T12:44:00.000000Z,None,https://ar.rdcpix.com/24eeaf81225afb464d089f9e...,1,N/A,https://www.realtor.com/apartments/10824-Big-B...


In [10]:
# Assuming your dataframe is called 'df'
# Group by the three columns and calculate the summary statistics for Monthly Rent
rent_summary = df_rent.groupby(['Property Type', 'Beds', 'Baths'])['Monthly Rent v2'].agg([
    ('Count', 'count'),
    ('Min Rent', 'min'),
    ('Median Rent', 'median'),
    ('Max Rent', 'max')
]).reset_index()

# Display the results
print(rent_summary)

    Property Type  Beds Baths  Count Min Rent Median Rent Max Rent
0       apartment   2.0     1      1     1595      1595.0     1595
1          condos   1.0     1      1     1095      1095.0     1095
2          condos   2.0     1      2     1099      1297.0     1495
3          condos   2.0     2      2     1675      2037.5     2400
4          condos   3.0     2      2     1895      1922.5     1950
5   single_family   2.0     1      2     1395      1472.5     1550
6   single_family   2.0   1.5      1     4500      4500.0     4500
7   single_family   3.0     1      1     2750      2750.0     2750
8   single_family   3.0   1.5      1     2570      2570.0     2570
9   single_family   3.0     2      2     2000      3950.0     5900
10  single_family   3.0   2.5      2     2850      2875.0     2900
11  single_family   3.0     3      1     3000      3000.0     3000
12  single_family   4.0     3      1     3895      3895.0     3895
13      townhomes   3.0   1.5      1     2395      2395.0     

In [5]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
import requests
import pandas as pd
import json

# Set page title and description
st.title('Realtor.com Rental Property Finder')
st.write('Enter a zip code to find available rental properties in that area.')

# Create zip code input field
zip_code = st.text_input('Enter zip code:', placeholder='e.g., 90210')

# Optional: Add validation for zip code format
if zip_code and not (zip_code.isdigit() and len(zip_code) == 5):
    st.warning('Please enter a valid 5-digit zip code')

# Create a search button
search_button = st.button('Search Properties')

# Only process when the search button is clicked and zip code is valid
if search_button and zip_code and zip_code.isdigit() and len(zip_code) == 5:
    with st.spinner('Fetching rental properties...'):
        # Call API function
        results = 'hello world'

        df = df_rent

# Display as interactive table
        st.subheader('Available Rental Properties')
        st.dataframe(df)
                
        # Optional: Add filters
        st.subheader('Filter Results')
        min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
        max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
            

In [ ]:
import streamlit as st
import requests
import pandas as pd
import json

# Set page title and description
st.title('Realtor.com Rental Property Finder')
st.write('Enter a zip code to find available rental properties in that area.')

# Create zip code input field
zip_code = st.text_input('Enter zip code:', placeholder='e.g., 90210')

# Optional: Add validation for zip code format
if zip_code and not (zip_code.isdigit() and len(zip_code) == 5):
    st.warning('Please enter a valid 5-digit zip code')

# Create a search button
search_button = st.button('Search Properties')

# Function to call Realtor.com API
#def get_rental_properties(zip_code):
#    # Replace this with your actual API request code
#    url = f"https://realtor.p.rapidapi.com/properties/v2/list-for-rent"
#    
#    querystring = {
#        "postal_code": zip_code,
#        "limit": "20",
#        "offset": "0",
#        "sort": "relevance"
#    }
#    
#    headers = {
#        "X-RapidAPI-Key": st.secrets["RAPIDAPI_KEY"],  # Store API key in Streamlit secrets
#        "X-RapidAPI-Host": "realtor.p.rapidapi.com"
#    }
#    
#    try:
#        response = requests.get(url, headers=headers, params=querystring)
#        response.raise_for_status()  # Raise exception for 4XX/5XX responses
#        return response.json()
#    except requests.exceptions.RequestException as e:
#        st.error(f"API Error: {e}")
#        return None

# Only process when the search button is clicked and zip code is valid
if search_button and zip_code and zip_code.isdigit() and len(zip_code) == 5:
    with st.spinner('Fetching rental properties...'):
        # Call API function
        #results = get_rental_properties(zip_code)
        results = 'hello world'
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
        results = get_rental_properties(zip_code)
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
        results = get_rental_properties(zip_code)
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
                # Apply filters
                filtered_df = df[
                    (df['Beds'] >= min_beds) & 
                    (df['Price'] <= max_price)
                ]
                
                st.subheader('Filtered Properties')
                st.dataframe(filtered_df)
                
                # Visualization
                st.subheader('Price Distribution')
                if 'Price' in df.columns and df['Price'].notna().any():
                    # Convert prices to numeric if they're not already
                    if not pd.api.types.is_numeric_dtype(df['Price']):
                        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
                    
                    st.bar_chart(df['Price'].value_counts().sort_index())
        else:
            st.error("Failed to retrieve data from Realtor.com API")

# Add instructions
with st.expander("How to use this app"):
    st.write("""
    1. Enter a valid 5-digit zip code in the input field
    2. Click the 'Search Properties' button
    3. View the retrieved rental properties
    4. Use the filters to narrow down results by bedrooms and price
    """)

# Note about API key
st.sidebar.info("""
**Note:** This app requires a valid Realtor.com API key.
Store your API key in Streamlit secrets.
""")